<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modelling-Data" data-toc-modified-id="Modelling-Data-1">Modelling Data</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2">Data</a></span></li><li><span><a href="#Restaurant-Density" data-toc-modified-id="Restaurant-Density-3">Restaurant Density</a></span></li><li><span><a href="#Joining-restaurant-data-and-demographic-data-together" data-toc-modified-id="Joining-restaurant-data-and-demographic-data-together-4">Joining restaurant data and demographic data together</a></span></li><li><span><a href="#Turning-FSA-values-into-dummy-variables" data-toc-modified-id="Turning-FSA-values-into-dummy-variables-5">Turning FSA values into dummy variables</a></span></li><li><span><a href="#Turning-categories-into-dummy-variables" data-toc-modified-id="Turning-categories-into-dummy-variables-6">Turning categories into dummy variables</a></span></li><li><span><a href="#Removing-non-numeric-columns" data-toc-modified-id="Removing-non-numeric-columns-7">Removing non-numeric columns</a></span><ul class="toc-item"><li><span><a href="#Changing-Hours-column" data-toc-modified-id="Changing-Hours-column-7.1">Changing Hours column</a></span></li></ul></li><li><span><a href="#Export-Data" data-toc-modified-id="Export-Data-8">Export Data</a></span></li></ul></div>

# Modelling Data

In this notebook, I will be combing the data, turning some variables into dummy variables, and getting the data ready for modelling. Namely, the tasks are:

- Add a `restaurant_density` metric into `demo_data`
- Join `rest_data` and `demo_data` together
- Turn `FSA` column in `rest_data` into dummy variables
- Remove any non-numeric columns

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data 

In [2]:
demo_data = pd.read_csv("cleaned_demographic_data.csv")
rest_data = pd.read_csv("cleaned_rest_data.csv")
print(f"Shape of rest_data: {rest_data.shape}")
print(f"Shape of demo_data: {demo_data.shape}")

Shape of rest_data: (7202, 15)
Shape of demo_data: (98, 29)


In [3]:
demo_data.head()

,Unnamed: 0,FSA,Visible_Minority_Population_Pct,South_Asian_Pct,Chinese_Pct,Black_Pct,Filipino_Pct,Latin American_Pct,Arab_Pct,South_East_Asian_Pct,...,Age:60plus,Median Income,Commute:Drive,Commute:Walk_Bus_Bike,NonImmigrants,Immigrants,House_Owner_Pct,House_Renter_Pct,Median_Monthly_Rent,Median_Monthly_Buy
0,0,M5V,20745.0,4395.0,6290.0,2560.0,1020.0,1355.0,1000.0,605.0,...,3045.0,69202.0,8540.0,25265.0,28920.0,16245.0,13265.0,17455.0,1715.0,1886.0
1,1,M5T,10455.0,765.0,6045.0,880.0,450.0,330.0,290.0,615.0,...,3535.0,48942.0,1255.0,6935.0,8500.0,7610.0,2510.0,6035.0,1101.0,1410.0
2,2,M6J,9055.0,900.0,4225.0,1100.0,440.0,615.0,170.0,335.0,...,5210.0,57063.0,5445.0,12115.0,20335.0,10225.0,7355.0,7760.0,1402.0,1683.0
3,3,M6G,7605.0,740.0,2415.0,1120.0,570.0,720.0,155.0,355.0,...,7190.0,59052.0,4870.0,11030.0,20505.0,9780.0,6640.0,6775.0,1279.0,1328.0
4,4,M5S,6450.0,1045.0,2980.0,355.0,210.0,225.0,265.0,145.0,...,3015.0,66185.0,1365.0,5365.0,7220.0,5120.0,3025.0,4930.0,1686.0,1442.0


In [4]:
rest_data.head()

,Unnamed: 0,name,review_count,categories,FSA,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,price_range,take_out,success
0,0,Bakery Gateau,8,Bakeries,M3B,11.0,11.0,11.0,11.0,11.0,11.0,11.0,2,1,1
1,1,Bolt Fresh Bar,57,Vegetarian/Vegan,M6J,13.0,13.0,13.0,13.0,13.0,12.0,12.0,2,1,0
2,2,The Steady Cafe & Bar,29,Coffee/Tea,M6H,0.0,9.0,9.0,9.0,7.0,8.0,5.0,2,0,0
3,3,Mad Crush Wine Bar,9,Breakfast/Brunch,M6G,0.0,0.0,0.0,16.0,16.0,16.0,0.0,3,1,0
4,4,Burrito Bandidos,43,Mexican,M4L,12.0,12.0,12.0,12.0,12.0,11.0,10.0,2,1,1


# Restaurant Density

A metric that I would like to add is the density of restaurants within a postal code. For example, it would be a good statistic to include if some FSA's had only one or two restaurants, whereas some FSA's has more than one restaurant. The metric I will use is:

$$
\text{Restaurant Density} = \frac{\text{Number of Restaurant}}{\text{1 FSA Value}}
$$

Therefore, a FSA with a restaurant density of 3 means that there are 3 restaurants within the FSA.

In [5]:
rest_data.head()

,Unnamed: 0,name,review_count,categories,FSA,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,price_range,take_out,success
0,0,Bakery Gateau,8,Bakeries,M3B,11.0,11.0,11.0,11.0,11.0,11.0,11.0,2,1,1
1,1,Bolt Fresh Bar,57,Vegetarian/Vegan,M6J,13.0,13.0,13.0,13.0,13.0,12.0,12.0,2,1,0
2,2,The Steady Cafe & Bar,29,Coffee/Tea,M6H,0.0,9.0,9.0,9.0,7.0,8.0,5.0,2,0,0
3,3,Mad Crush Wine Bar,9,Breakfast/Brunch,M6G,0.0,0.0,0.0,16.0,16.0,16.0,0.0,3,1,0
4,4,Burrito Bandidos,43,Mexican,M4L,12.0,12.0,12.0,12.0,12.0,11.0,10.0,2,1,1


In [6]:
# Iterating over one FSA value in demo_data (0th index FSA)
demo_data['FSA'][0]
# Counting how many times that single iteration appears in rest_data (0th index FSA)
len(rest_data[rest_data['FSA'] == demo_data['FSA'][0]])



493

The first FSA value in the demo_data occurs 493 times. I want to create a new dataframe where it just has FSA values and its count.

In [7]:
# List of FSA
fsa_list = demo_data['FSA'].value_counts().index
# Populate this list
fsa_value_list = []

# Loop over fsa_list to find density for each value
for fsa_value in fsa_list:
    count = len(rest_data[rest_data['FSA'] == fsa_value])
    fsa_value_list.append(count)

# Create dataframe of these values
fsa_density = pd.DataFrame(fsa_value_list, columns = ['Restaurant_Density']).T
fsa_density.columns = fsa_list

# Look at it 
fsa_density = fsa_density.T.reset_index()

fsa_density

,index,Restaurant_Density
0,M5G,152
1,M1B,17
2,M6K,151
3,M3A,8
4,M5S,237
...,...,...
93,M4S,111
94,M1W,27
95,M3N,12
96,M9N,22


Now, we need to add it to demo_data

In [8]:
demo_data = demo_data.merge(fsa_density, left_on = "FSA", right_on = "index").drop(columns = ['index', 'Unnamed: 0'])
demo_data.head()

,FSA,Visible_Minority_Population_Pct,South_Asian_Pct,Chinese_Pct,Black_Pct,Filipino_Pct,Latin American_Pct,Arab_Pct,South_East_Asian_Pct,West_Asian_Pct,...,Median Income,Commute:Drive,Commute:Walk_Bus_Bike,NonImmigrants,Immigrants,House_Owner_Pct,House_Renter_Pct,Median_Monthly_Rent,Median_Monthly_Buy,Restaurant_Density
0,M5V,20745.0,4395.0,6290.0,2560.0,1020.0,1355.0,1000.0,605.0,1030.0,...,69202.0,8540.0,25265.0,28920.0,16245.0,13265.0,17455.0,1715.0,1886.0,493
1,M5T,10455.0,765.0,6045.0,880.0,450.0,330.0,290.0,615.0,190.0,...,48942.0,1255.0,6935.0,8500.0,7610.0,2510.0,6035.0,1101.0,1410.0,432
2,M6J,9055.0,900.0,4225.0,1100.0,440.0,615.0,170.0,335.0,150.0,...,57063.0,5445.0,12115.0,20335.0,10225.0,7355.0,7760.0,1402.0,1683.0,325
3,M6G,7605.0,740.0,2415.0,1120.0,570.0,720.0,155.0,355.0,155.0,...,59052.0,4870.0,11030.0,20505.0,9780.0,6640.0,6775.0,1279.0,1328.0,300
4,M5S,6450.0,1045.0,2980.0,355.0,210.0,225.0,265.0,145.0,205.0,...,66185.0,1365.0,5365.0,7220.0,5120.0,3025.0,4930.0,1686.0,1442.0,237


# Joining restaurant data and demographic data together

In [9]:
print("Before joining datasets")
print("")
print(f"Shape of rest_data: {rest_data.shape}")
print(f"Shape of demo_data: {demo_data.shape}")

Before joining datasets

Shape of rest_data: (7202, 15)
Shape of demo_data: (98, 29)


In [10]:
model_data = rest_data.merge(demo_data, on='FSA').drop(columns = ['Unnamed: 0'])

In [11]:
print("After joining datasets")
print("")
print(f"Shape of model_data: {model_data.shape}")

After joining datasets

Shape of model_data: (7202, 42)


In [12]:
model_data

,name,review_count,categories,FSA,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,...,Median Income,Commute:Drive,Commute:Walk_Bus_Bike,NonImmigrants,Immigrants,House_Owner_Pct,House_Renter_Pct,Median_Monthly_Rent,Median_Monthly_Buy,Restaurant_Density
0,Bakery Gateau,8,Bakeries,M3B,11.0,11.0,11.0,11.0,11.0,11.0,...,80521.0,3675.0,1365.0,6670.0,6265.0,3955.0,1030.0,1496.0,1381.0,32
1,Better than Yia Yia's,5,West Asian,M3B,12.0,12.0,12.0,12.0,12.0,12.0,...,80521.0,3675.0,1365.0,6670.0,6265.0,3955.0,1030.0,1496.0,1381.0,32
2,Spoon And Fork Plus,165,South East Asian,M3B,10.5,10.5,10.5,10.5,11.5,11.5,...,80521.0,3675.0,1365.0,6670.0,6265.0,3955.0,1030.0,1496.0,1381.0,32
3,California Sandwiches,36,Breakfast/Brunch,M3B,11.0,11.0,11.0,11.0,11.0,11.0,...,80521.0,3675.0,1365.0,6670.0,6265.0,3955.0,1030.0,1496.0,1381.0,32
4,Mamma's Pizza,11,Non-Visible Minority Foods,M3B,12.0,12.0,12.0,12.0,11.0,11.0,...,80521.0,3675.0,1365.0,6670.0,6265.0,3955.0,1030.0,1496.0,1381.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7197,Prague Restaurant,14,Non-Visible Minority Foods,M1G,0.0,4.0,5.0,5.0,5.0,10.0,...,44277.0,6865.0,4375.0,11995.0,16575.0,4810.0,5010.0,981.0,1434.0,5
7198,Spice Shack,3,West Asian,M1G,5.5,5.5,5.5,5.5,11.0,11.0,...,44277.0,6865.0,4375.0,11995.0,16575.0,4810.0,5010.0,981.0,1434.0,5
7199,Panchvati Supermarket,3,Specialty,M1G,11.5,11.5,11.5,11.5,11.5,11.5,...,44277.0,6865.0,4375.0,11995.0,16575.0,4810.0,5010.0,981.0,1434.0,5
7200,Eli's Table,89,West Asian,L6A,11.0,11.0,11.0,11.0,11.0,11.0,...,60091.0,35280.0,6585.0,43840.0,40375.0,22485.0,2345.0,1687.0,2077.0,1


# Turning FSA values into dummy variables

In [13]:
model_data = pd.concat([model_data, pd.get_dummies(model_data['FSA'])], axis = 1)

In [14]:
print("After Turning FSA Values into dummy variables")
print("")
print(f"Shape of model_data: {model_data.shape}")

After Turning FSA Values into dummy variables

Shape of model_data: (7202, 140)


# Turning categories into dummy variables

In [15]:
model_data = pd.concat([model_data, pd.get_dummies(model_data['categories'])], axis = 1)

In [16]:
# print("After turning categories into dummy variables")
# print("")
# print(f"Shape of model_data: {model_data.shape}")

# Removing non-numeric columns

In [17]:
model_data = model_data.drop(columns = 'categories')
model_data.head()

,name,review_count,FSA,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,...,Desserts,Japanese,Korean,Mexican,Misc,Non-Visible Minority Foods,South East Asian,Specialty,Vegetarian/Vegan,West Asian
0,Bakery Gateau,8,M3B,11.0,11.0,11.0,11.0,11.0,11.0,11.0,...,0,0,0,0,0,0,0,0,0,0
1,Better than Yia Yia's,5,M3B,12.0,12.0,12.0,12.0,12.0,12.0,12.0,...,0,0,0,0,0,0,0,0,0,1
2,Spoon And Fork Plus,165,M3B,10.5,10.5,10.5,10.5,11.5,11.5,10.5,...,0,0,0,0,0,0,1,0,0,0
3,California Sandwiches,36,M3B,11.0,11.0,11.0,11.0,11.0,11.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Mamma's Pizza,11,M3B,12.0,12.0,12.0,12.0,11.0,11.0,12.0,...,0,0,0,0,0,1,0,0,0,0


## Changing Hours column

In [18]:
# Adding an average hours column
model_data['avg_hours_in_week'] = round((model_data['Monday'] + model_data['Tuesday'] + model_data['Wednesday'] + \
                                  model_data['Thursday'] + model_data['Friday'] + model_data['Saturday'] + \
                                  model_data['Sunday']) / 7, 3)

# Dropping hours per day
model_data = model_data.drop(columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# Export Data

In [19]:
model_data.head()

,name,review_count,FSA,price_range,take_out,success,Visible_Minority_Population_Pct,South_Asian_Pct,Chinese_Pct,Black_Pct,...,Japanese,Korean,Mexican,Misc,Non-Visible Minority Foods,South East Asian,Specialty,Vegetarian/Vegan,West Asian,avg_hours_in_week
0,Bakery Gateau,8,M3B,2,1,1,6495.0,1070.0,3310.0,185.0,...,0,0,0,0,0,0,0,0,0,11.000
1,Better than Yia Yia's,5,M3B,2,1,1,6495.0,1070.0,3310.0,185.0,...,0,0,0,0,0,0,0,0,1,12.000
2,Spoon And Fork Plus,165,M3B,2,1,0,6495.0,1070.0,3310.0,185.0,...,0,0,0,0,0,1,0,0,0,10.786
3,California Sandwiches,36,M3B,2,1,1,6495.0,1070.0,3310.0,185.0,...,0,0,0,0,0,0,0,0,0,9.429
4,Mamma's Pizza,11,M3B,2,1,0,6495.0,1070.0,3310.0,185.0,...,0,0,0,0,1,0,0,0,0,11.714


In [20]:
model_data.to_csv('model_data.csv')